In [1]:
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.set_num_threads(8)

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "FMNIST",
            'random_class_idx' : False,
            'method': "vanilla",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 50,
            # 'per_task_examples': np.inf,
            'per_task_examples': 200000,
            'per_task_memory_examples': 64,
            'batch_size_train': 256,
            'batch_size_memory': 256,
            # 'batch_size_memory': 256,
            'batch_size_validation': 256,
            'tau': 1.5,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.01,
            'learning_rate_decay_epoch': [30, 50, 70, 90],
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:6' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.0,
            'lambda': 0.0,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=CIFAR10/seed=10_epoch=50_lr=0.01_alpha=0.0_tau=1.5


In [2]:
from datasets import MNIST, FashionMNIST, BiasedMNIST, CIFAR10, CIFAR100
if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                joint = (params['method'] == "joint"),
                                random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        joint = (params['method'] == "joint"),
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 28, 28)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)


[0 1 2 3 4 5 6 7 8 9]
Files already downloaded and verified
Files already downloaded and verified


In [3]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2
from backbones import MLP2Layers2

from backbones.resnet import ResNet18


# backbone = ResNet18(
#     input_dim=input_dim, 
#     output_dim=num_classes,
#     class_idx=class_idx,
#     config=params
#     ).to(params['device'])

backbone = ResNet18(
    input_dim=input_dim, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])

algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

modify resnet for cifar


In [4]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [5]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.821, 'loss': 0.0015542374849319457, 'std': 0.09600000000000003, 'EER': -1}
[2] Eval metrics for task 1 >> {'accuracy': 0.8505, 'loss': 0.0013441222906112672, 'std': 0.08650000000000002, 'EER': -1}
[3] Eval metrics for task 1 >> {'accuracy': 0.8835, 'loss': 0.001098384290933609, 'std': 0.04150000000000004, 'EER': -1}
[4] Eval metrics for task 1 >> {'accuracy': 0.905, 'loss': 0.0009727678298950195, 'std': 0.013000000000000012, 'EER': -1}
[5] Eval metrics for task 1 >> {'accuracy': 0.9145, 'loss': 0.0008592504635453224, 'std': 0.04149999999999998, 'EER': -1}
[6] Eval metrics for task 1 >> {'accuracy': 0.927, 'loss': 0.0007724330499768257, 'std': 0.023999999999999966, 'EER': -1}
[7] Eval metrics for task 1 >> {'accuracy': 0.872, 'loss': 0.0012186369150877, 'std': 0.10999999999999999, 'EER': -1}
[8] Eval metrics for task 1 >> {'accuracy': 0.9239999999999999, 'loss': 0.000810222119092941

In [6]:
metric_manager_callback.meters['EER'].get_eer()

[0.0, 0.019000000000000017, 0.05088888888888891, 0.277625, 0.19852]

In [7]:
metric_manager_callback.meters['std'].get_std()

[0.04199999999999998,
 0.13833835332256922,
 0.08157954536660661,
 0.3491230979181985,
 0.2546220925214464]

In [8]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.943, 0.   , 0.   , 0.   , 0.   ],
       [0.591, 0.629, 0.   , 0.   , 0.   ],
       [0.558, 0.41 , 0.477, 0.   , 0.   ],
       [0.334, 0.07 , 0.182, 0.936, 0.   ],
       [0.205, 0.142, 0.216, 0.285, 0.832]])

In [9]:
metric_manager_callback.meters['accuracy'].compute_overall()

[0.9430000000000001, 0.61, 0.4816666666666667, 0.38025, 0.3357]

In [10]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:0.5501233333333333
EER:0.10920677777777779
std:0.17313261782576414
